In [ ]:
#pip install slackclient
#pip install kafka-python
import os
import time
from slackclient import SlackClient

token = 'Your token here' #(https://api.slack.com/docs/oauth-test-tokens)
sc = SlackClient(token)


channels = [channel_dict['id'] for channel_dict in sc.api_call("channels.list")['channels']]
print channels
    

In [ ]:
%%writefile example.py
#!/home/kevin/slackpstone/bin/python

# Example from https://github.com/dpkp/kafka-python/blob/master/example.py
import threading, logging, time

from kafka import KafkaConsumer, KafkaProducer


class Producer(threading.Thread):
    daemon = True

    def run(self):
        producer = KafkaProducer(bootstrap_servers='localhost:9092')

        while True:
            producer.send('my-topic', b"test")
            producer.send('my-topic', b"\xc2Hola, mundo!")
            time.sleep(1)


class Consumer(threading.Thread):
    daemon = True

    def run(self):
        consumer = KafkaConsumer(bootstrap_servers='localhost:9092',
                                 auto_offset_reset='earliest')
        consumer.subscribe(['my-topic'])

        for message in consumer:
            print (message)


def main():
    threads = [
        Producer(),
        Consumer()
    ]

    for t in threads:
        t.start()

    time.sleep(20)

if __name__ == "__main__":
    logging.basicConfig(
        format='%(asctime)s.%(msecs)s:%(name)s:%(thread)d:%(levelname)s:%(process)d:%(message)s',
        level=logging.INFO
        )
main()

In [ ]:
!python example.py

In [ ]:
# Reading from channel history

output_example =  open('slackpstone-channel-output.txt', 'w')

for channel in channels:
    channel_history = sc.api_call("channels.history", channel=channel, count="100000")
    for message_dict in channel_history['messages']:
        if 'user' in message_dict:
            output_example.write('{}\t{}\t{}\n'.format(
                message_dict['text'].replace('\n','').encode('utf-8'),
                    message_dict['user'], message_dict['ts']))
output_example.close()

In [ ]:
# Example output
c = 0
with open('slackpstone-channel-output.txt', 'r') as f:
    for line in f:
        if c<10:
            print line.strip().split('\t')
            c += 1
        else:
            break

In [ ]:
%%writefile slack_example.py
#!/home/kevin/slackpstone/bin/python

# Integrating slack api and kafka
from slackclient import SlackClient
from kafka import KafkaConsumer, KafkaProducer
import threading, logging, time

producer = KafkaProducer(bootstrap_servers='localhost:9092')
c = 0
token = 'Your Token'
sc = SlackClient(token)
channels = [channel_dict['id'] for channel_dict in sc.api_call("channels.list")['channels']]
for channel in channels:
    channel_history = sc.api_call("channels.history", channel=channel, count="100000")
    for message_dict in channel_history['messages']:
        if 'user' in message_dict:
            message = '{}\t{}\t{}\n'.format(
                message_dict['text'].replace('\n','').encode('utf-8'),
                    message_dict['user'], message_dict['ts'])
            producer.send(channel, message)
            c += 1
print 'Sent {} messages'.format(c)

In [ ]:
!python slack_example.py